learn some pydantic first

3.5 sucks at function calling ),: gpt4 better

In [1]:
import pydantic
from pydantic import BaseModel, Field
pydantic.__version__

'2.6.0'

In [2]:
class User(BaseModel):
    id: int
    name: str

test = {
    "id": 1,
    "name": "John Doe"

}

test_fake = {
    "id": "1",
    "name": "John Doe",
    "height": 1.8   

}

user = User(**test)
user_fake = User(**test_fake)

print(user), print(user_fake), isinstance(user_fake.id, int)


id=1 name='John Doe'
id=1 name='John Doe'


(None, None, True)

In [3]:
from pydantic import conlist

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip: str

class Employee(BaseModel):
    id: int
    name: str
    address: Address
    email: str

class Owner(BaseModel):
    id: int
    name: str
    address: Address
    email: str

class Restuarant(BaseModel):
    id: int
    name: str
    owner: Owner
    employees: conlist(Employee, min_length=2)


In [4]:
Restuarant.model_json_schema()

{'$defs': {'Address': {'properties': {'street': {'title': 'Street',
     'type': 'string'},
    'city': {'title': 'City', 'type': 'string'},
    'state': {'title': 'State', 'type': 'string'},
    'zip': {'title': 'Zip', 'type': 'string'}},
   'required': ['street', 'city', 'state', 'zip'],
   'title': 'Address',
   'type': 'object'},
  'Employee': {'properties': {'id': {'title': 'Id', 'type': 'integer'},
    'name': {'title': 'Name', 'type': 'string'},
    'address': {'$ref': '#/$defs/Address'},
    'email': {'title': 'Email', 'type': 'string'}},
   'required': ['id', 'name', 'address', 'email'],
   'title': 'Employee',
   'type': 'object'},
  'Owner': {'properties': {'id': {'title': 'Id', 'type': 'integer'},
    'name': {'title': 'Name', 'type': 'string'},
    'address': {'$ref': '#/$defs/Address'},
    'email': {'title': 'Email', 'type': 'string'}},
   'required': ['id', 'name', 'address', 'email'],
   'title': 'Owner',
   'type': 'object'}},
 'properties': {'id': {'title': 'Id', 'ty

In [5]:
res = Restuarant(
    id=1,
    name="The Palace",
    owner=Owner(
        id=1,
        name="John Doe",
        address=Address(
            street="1234 Elm St",
            city="Springfield",
            state="IL",
            zip="62701"
        ),
        email="email.com",
    ),
    employees=[
        Employee(
            id=1,
            name="Jane Doe",
            address=Address(
                street="1234 Elm St",
                city="Springfield",
                state="IL",
                zip="62701"
            ),
            email="email.com"
        ),
        Employee(
            id=2,
            name="Jim Doe",
            address=Address(
                street="1234 Elm St",
                city="Springfield",
                state="IL",
                zip="62701"
            ),
            email="email.com"
        )
    ]




)

In [6]:
for emp in res.employees:
    print(emp.name, emp.address.city, emp.email)
    


Jane Doe Springfield email.com
Jim Doe Springfield email.com


In [7]:
from pydantic import EmailStr, field_validator

#field validators.. validates JUST a single field

class Owner(BaseModel):
    name: str
    email: EmailStr

    @field_validator("name")
    @classmethod
    def name_must_contain_space(cls, v):
        if " " not in v:
            raise ValueError("Must contain a space you idiot")
        return v.title()
    


try:
    owner_instance = Owner(name="john doe", email="johndoe@gmail.com")
    print(owner_instance)
except Exception as e:
    print(e)


name='John Doe' email='johndoe@gmail.com'


In [8]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
wes = Person(first_name="Wes", last_name="Stone", age=40)
wes

Person(first_name='Wes', last_name='Stone', age=40)

In [9]:
from pydantic import ValidationError

try:
    Person(first_name="Wes", last_name="Stone", age="40")
except ValidationError as e:
    print(e)


wes = Person(first_name="notWes", last_name="Stone", age=40)

wes.first_name = "Wes"
wes.first_name

'Wes'

In [10]:
try:
    Person(first_name="Wes", last_name=100, age="junk")
except ValidationError as e:
    exceptions = e

exceptions.errors(), exceptions.json()


([{'type': 'string_type',
   'loc': ('last_name',),
   'msg': 'Input should be a valid string',
   'input': 100,
   'url': 'https://errors.pydantic.dev/2.6/v/string_type'},
  {'type': 'int_parsing',
   'loc': ('age',),
   'msg': 'Input should be a valid integer, unable to parse string as an integer',
   'input': 'junk',
   'url': 'https://errors.pydantic.dev/2.6/v/int_parsing'}],
 '[{"type":"string_type","loc":["last_name"],"msg":"Input should be a valid string","input":100,"url":"https://errors.pydantic.dev/2.6/v/string_type"},{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"junk","url":"https://errors.pydantic.dev/2.6/v/int_parsing"}]')

In [11]:
data = {
    "first_name": "Wes",
    "last_name": "Stone",
    "age": 40
}

p = Person(**data)
p

Person(first_name='Wes', last_name='Stone', age=40)

In [12]:
data = '''{
    "first_name": "Wes",
    "last_name": "Stone",
    "age": 40
}
'''

p = Person.model_validate_json(data)

p

Person(first_name='Wes', last_name='Stone', age=40)

In [13]:
try:
    p = Person(age=40)
except ValidationError as e:
    print(e)


2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 40}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
last_name
  Field required [type=missing, input_value={'age': 40}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing


In [14]:
class Person(BaseModel):
    first_name: str = "Wes"
    last_name: str = "Stone"
    age: int = 20


Person.model_fields, Person()


({'first_name': FieldInfo(annotation=str, required=False, default='Wes'),
  'last_name': FieldInfo(annotation=str, required=False, default='Stone'),
  'age': FieldInfo(annotation=int, required=False, default=20)},
 Person(first_name='Wes', last_name='Stone', age=20))

In [15]:
class Person(BaseModel):
    first_name: str | None = None
    last_name: str
    age: int

p = Person(last_name="Stone", age=40)
p

Person(first_name=None, last_name='Stone', age=40)

In [20]:
from pydantic import Field

data = {
    "id": 100,
    "First Name": "Wes",
    "LASTNAME": "Stone",
    "age in years": 40
}

In [23]:
class Person(BaseModel):
    id_: int = Field(alias="id")
    first_name: str = Field(alias="First Name")
    last_name: str = Field(alias="LASTNAME")
    age: int = Field(alias="age in years")

p = Person(**data)
p


Person(id_=100, first_name='Wes', last_name='Stone', age=40)

In [35]:
import instructor
from openai import OpenAI
from pydantic import BaseModel

# This enables response_model keyword
# from client.chat.completions.create
client = instructor.patch(OpenAI())

class UserDetail(BaseModel):
    name: str
    age: int

user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Wes is 24 years old."},
    ]
)

print(UserDetail.model_json_schema())
assert isinstance(user, UserDetail)
assert user.name == "Wes"
assert user.age == 24

{'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}}, 'required': ['name', 'age'], 'title': 'UserDetail', 'type': 'object'}


In [36]:
user: UserDetail = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Jason is 25 years old"},
    ]
)

from openai.types.chat.chat_completion import ChatCompletion

assert isinstance(user._raw_response, ChatCompletion)

In [37]:
user._raw_response

ChatCompletion(id='chatcmpl-8nI4SqMnZHQ70yv7bMv3V3RGeuxaO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "name": "Jason",\n  "age": 25\n}', name='UserDetail'), tool_calls=None))], created=1706758228, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=72, total_tokens=88))

In [51]:
import enum

class Labels(str, enum.Enum):
    good = "good"
    bad = "bad"
    neutral = "neutral"



class Sentiment(BaseModel):
    label: Labels
    

Sentiment.model_json_schema()

{'$defs': {'Labels': {'enum': ['good', 'bad', 'neutral'],
   'title': 'Labels',
   'type': 'string'}},
 'properties': {'label': {'$ref': '#/$defs/Labels'}},
 'required': ['label'],
 'title': 'Sentiment',
 'type': 'object'}

In [55]:
UserDetail = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=Sentiment,
    messages=[
        {"role": "user", "content": "That was so terrible and I hated it"},
    ]
)


In [57]:
user

Sentiment(label=<Labels.bad: 'bad'>)

In [58]:
import enum
from pydantic import BaseModel

class Labels(str, enum.Enum):
    """Enumeration for single-label text classification."""
    SPAM = "spam"
    NOT_SPAM = "not_spam"

class SinglePrediction(BaseModel):
    """
    Class for a single class label prediction.
    """
    class_label: Labels

In [59]:
from openai import OpenAI
import instructor

# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.patch(OpenAI())

def classify(data: str) -> SinglePrediction:
    """Perform single-label classification on the input text."""
    return client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        response_model=SinglePrediction,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following text: {data}",
            },
        ],
    )  # type:

In [62]:
test = classify("Give me your money")

In [69]:
test.class_label.value

'spam'

In [72]:
from typing import List
# Define Enum class for multiple labels
class MultiLabels(str, enum.Enum):
    TECH_ISSUE = "tech_issue"
    BILLING = "billing"
    GENERAL_QUERY = "general_query"

# Define the multi-class prediction model
class MultiClassPrediction(BaseModel):
    """
    Class for a multi-class label prediction.
    """
    class_labels: List[MultiLabels]

MultiClassPrediction.model_json_schema()

{'$defs': {'MultiLabels': {'enum': ['tech_issue', 'billing', 'general_query'],
   'title': 'MultiLabels',
   'type': 'string'}},
 'description': 'Class for a multi-class label prediction.',
 'properties': {'class_labels': {'items': {'$ref': '#/$defs/MultiLabels'},
   'title': 'Class Labels',
   'type': 'array'}},
 'required': ['class_labels'],
 'title': 'MultiClassPrediction',
 'type': 'object'}

In [81]:
def multi_classify(data: str) -> MultiClassPrediction:
    """Perform multi-label classification on the input text."""
    UserDetail = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        response_model=MultiClassPrediction,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following support ticket: {data}",
            },
        ],
    )  # type: ignore
    return UserDetail

test = multi_classify("I am having trouble with my internet")

In [82]:
test

MultiClassPrediction(class_labels=[<MultiLabels.TECH_ISSUE: 'tech_issue'>, <MultiLabels.BILLING: 'billing'>, <MultiLabels.GENERAL_QUERY: 'general_query'>])

In [108]:
class Risks(str, enum.Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"

class Risk_factors(str, enum.Enum):
    CYBER_SECURITY = "cyber_security"
    FINANCIAL = "financial"
    LEGAL = "legal"
    OPERATIONAL = "operational"
    REPUTATION = "reputation"
    COMPETITION = "competition"
    MARKET = "market"
    REGULATORY = "regulatory"
    SUPPLY_CHAIN = "supply_chain"
    TECHNOLOGY = "technology"
    ENVIRONMENTAL = "environmental"
    SOCIAL = "social"
    POLITICAL = "political"
    ECONOMIC = "economic"


class RiskPrediction(BaseModel):
    risk: Risks
    risk_factors: List[Risk_factors] = Field(..., description="List of risk factors", min_length=2, max_length=5)
    justification: str = Field(..., description="In depth explanation of the risk prediction", min_length=200)

RiskPrediction.model_json_schema()

{'$defs': {'Risk_factors': {'enum': ['cyber_security',
    'financial',
    'legal',
    'operational',
    'reputation',
    'competition',
    'market',
    'regulatory',
    'supply_chain',
    'technology',
    'environmental',
    'social',
    'political',
    'economic'],
   'title': 'Risk_factors',
   'type': 'string'},
  'Risks': {'enum': ['low', 'medium', 'high'],
   'title': 'Risks',
   'type': 'string'}},
 'properties': {'risk': {'$ref': '#/$defs/Risks'},
  'risk_factors': {'description': 'List of risk factors',
   'items': {'$ref': '#/$defs/Risk_factors'},
   'maxItems': 5,
   'minItems': 2,
   'title': 'Risk Factors',
   'type': 'array'},
  'justification': {'description': 'In depth explanation of the risk prediction',
   'minLength': 200,
   'title': 'Justification',
   'type': 'string'}},
 'required': ['risk', 'risk_factors', 'justification'],
 'title': 'RiskPrediction',
 'type': 'object'}

In [109]:
def get_classification(data: str) -> RiskPrediction:
    """Perform risk classification on the input text."""
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=RiskPrediction,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following risk: {data}",
            },
        ],
    )  # type:

In [110]:
test = get_classification("The company is facing a cyber security threat. They are asking ransom. My company is in cyber security business")


In [115]:
test.risk_factors[0].value

'cyber_security'

In [116]:
data = {
    "risk": test.risk.value,  # Convert enum to string
    "risk_factors": [factor.value for factor in test.risk_factors],  # Convert list of enums to list of strings
    "justification": test.justification
}

In [119]:
import pandas as pd

data
df = pd.DataFrame([data])
df

,risk,risk_factors,justification
0,high,"[cyber_security, reputation]",The company is facing a significant cyber secu...
